In [1]:
import pandas as pd
import sys
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from tsfresh.feature_extraction import MinimalFCParameters


import tsfresh
from tsfresh.feature_extraction import extract_features

from xgboost import XGBRegressor

In [2]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [12]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCounter
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_by_index_transformer import SelectByIndexTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_by_mask_transformer import SelectByMaskTransformer
from transformers.select_features_wrapper import SelectFeaturesWrapper
from transformers.my_turns_transformation import MyTurnsTransformation
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_by_index_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_by_mask_transformer)
importlib.reload(transformers.select_features_wrapper)
importlib.reload(transformers.my_turns_transformation)


import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [4]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')


In [5]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                    ('get_relavent_turns', SelectByIndexTransformer(train.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectByIndexTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectByMaskTransformer('is_player', target=DATA_NAME)),
                    ])

In [6]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
# p_n_turns = p_turns.loc[train.index.unique(),:]
# b_n_turns = b_turns.loc[train.index.unique(),:]
# n_games

In [13]:
t_turns = MyTurnsTransformation().transform(n_turns)

Feature Extraction: 100%|██████████| 20/20 [01:48<00:00,  5.42s/it]


In [14]:
t_turns

,a_turn_type_Challenge__mean,a_turn_type_Pass__sum_values,a_turn_type_Timeout__mean,a_Z__mean,p_points__sum_values,p_points__median,p_points__mean,p_points__root_mean_square,p_score__root_mean_square,p_score__maximum,...,p_.__median,p_move_len__mean,p_move_len__root_mean_square,p_jokers_num__sum_values,p_jokers_num__mean,b_turn_type_End__mean,b_e__standard_deviation,b_l__mean,b_move_len__sum_values,b_jokers_num__mean
1,0.0,0.0,0.00000,0.037037,429.0,25.0,30.642857,39.272855,294.421588,429.0,...,0.0,3.857143,4.191829,2.0,0.142857,0.000000,0.000000,0.0,60.0,0.000000
3,0.0,0.0,0.00000,0.037037,440.0,28.0,31.428571,37.743495,272.088221,440.0,...,0.0,4.357143,4.652188,1.0,0.071429,0.000000,0.000000,0.0,57.0,0.076923
4,0.0,0.0,0.00000,0.034483,119.0,5.5,8.500000,11.871334,89.362025,119.0,...,1.0,2.928571,3.218252,0.0,0.000000,0.000000,0.000000,0.0,76.0,0.066667
5,0.0,0.0,0.00000,0.032258,325.0,20.5,20.312500,23.618055,223.028726,325.0,...,0.0,3.375000,3.553168,0.0,0.000000,0.000000,0.249444,0.0,67.0,0.133333
6,0.0,0.0,0.00000,0.040000,378.0,28.5,31.500000,35.296364,210.361593,378.0,...,0.5,4.750000,4.804512,0.0,0.000000,0.076923,0.000000,0.0,66.0,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0.0,0.0,0.03125,0.031250,340.0,16.5,21.250000,30.035396,209.688906,350.0,...,0.0,3.500000,3.840573,1.0,0.062500,0.000000,0.000000,0.0,73.0,0.062500
72770,0.0,0.0,0.03125,0.031250,97.0,6.0,6.062500,8.821139,70.608958,107.0,...,1.0,3.187500,3.363406,1.0,0.062500,0.000000,0.242061,0.0,79.0,0.062500
72771,0.0,0.0,0.00000,0.032258,390.0,20.5,24.375000,30.661458,243.943129,390.0,...,1.0,4.250000,4.527693,0.0,0.000000,0.000000,0.000000,0.0,59.0,0.133333
72772,0.0,0.0,0.00000,0.033333,388.0,19.0,24.250000,30.636987,228.681410,388.0,...,0.0,3.562500,3.929058,2.0,0.125000,0.000000,0.000000,0.0,64.0,0.000000


In [15]:
t_turns.to_csv('../data/fs_turns.csv', index_label='game_id')

so now full pipeline is:

In [ ]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

features = np.load('../feature_selection_consts/columns.npy', allow_pickle=True)


prePipe = Pipeline([
                    ('get_relavent_turns', SelectByIndexTransformer(train.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectByIndexTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectByMaskTransformer('is_player', target=DATA_NAME)),
                    ])
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
t_turns = MyTurnsTransformation().transform(n_turns) # I want to add that to pre later
n_data = n_games.merge(t_turns, left_index=True, right_index=True)

featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('select_col', SelectTransformer(features)),
                       # add regressor                                              
                      ]

reg_pipe = Pipeline(featureTransformers)
trans = reg_pipe.transform(n_data)